[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_pdf_tools_intro.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Hello, Pdftools SDK!
Add a cover page from an image to a PDF.

In [ ]:
import io
from pdftools_sdk.image import Document as ImageDocument
from pdftools_sdk.pdf import Document as PdfDocument
from pdftools_sdk.image2_pdf import Converter
from pdftools_sdk.image2_pdf.profiles import Default
from pdftools_sdk.document_assembly import DocumentAssembler

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
cover_image_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/ideas.tiff'
cover_image = 'ideas.tiff'
prepare_file(cover_image_url, cover_image)
content_pdf_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/PdfPrimerWhitepaper.pdf'
content_pdf_path = 'PdfPrimerWhitepaper.pdf'
prepare_file(content_pdf_url, content_pdf_path)
output_path = 'MergedWithCover.pdf'

In [ ]:
def add_cover_to_pdf(cover_image: str, content_pdf_path: str, output_path: str):
    # Convert the image cover to a PDF (stored in memory)
    with io.BytesIO() as cover_stream:
        with io.FileIO(cover_image, 'rb') as image_stream:
            with ImageDocument.open(image_stream) as image_document:

                # Create the profile for converting the image to PDF
                profile = Default()

                # Convert image to PDF
                converter = Converter()
                converter.convert(image_document, cover_stream, profile)

        # Prepare the content PDF and merge with the cover
        with io.FileIO(content_pdf_path, 'rb') as content_pdf_stream:
            with PdfDocument.open(content_pdf_stream) as content_pdf_document:

                # Open output stream and append cover and content
                with io.FileIO(output_path, 'wb+') as output_stream:
                    with DocumentAssembler(output_stream, None, None) as assembler:
                        # Append cover page (convert from memory stream to PDF)
                        assembler.append(PdfDocument.open(io.BytesIO(cover_stream.getvalue())), 1, 1)

                        # Append the content PDF
                        assembler.append(content_pdf_document)

                        # Finalize the merged document
                        assembler.assemble()

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("<PDFSDK,V1,MGAASQD6L2JMQHL54PK08RQX8GG4SS0M8DAHVPH0VMP3NB8R9DUK>")

    # Optional: Set your proxy configuration
    # Sdk.set_proxy("http://myproxy:8080")
    
    add_cover_to_pdf(cover_image, content_pdf_path, output_path)

    print(f"Successfully created file {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")